In [93]:
!pip install kaggle -q
!pip install tensorflow-addons -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
!mkdir ~/.kaggle
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d williamscott701/memotion-dataset-7k

100% 695M/695M [00:05<00:00, 242MB/s]
100% 695M/695M [00:05<00:00, 137MB/s]


In [6]:
!unzip -qq /content/memotion-dataset-7k.zip

In [99]:
import re
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv1D, Embedding, GlobalAveragePooling1D 
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [47]:
df = pd.read_csv('/content/memotion_dataset_7k/labels.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df = df.drop(columns = ['text_ocr'])
df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral


In [48]:
df[df.isnull().any(axis=1)]

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
119,image_120.jpg,NaN,not_funny,general,not_offensive,not_motivational,positive
4799,image_4800.jpg,NaN,very_funny,general,slight,motivational,neutral
6781,image_6782.jpg,NaN,very_funny,twisted_meaning,not_offensive,not_motivational,positive
6784,image_6785.jpg,NaN,hilarious,general,not_offensive,not_motivational,positive
6786,image_6787.jpg,NaN,not_funny,not_sarcastic,very_offensive,motivational,positive


In [49]:
full_df = df.copy()
full_df.isnull().any()

image_name           False
text_corrected        True
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [50]:
clean_df = df.copy()
clean_df.dropna(inplace=True)
clean_df.isnull().any()

image_name           False
text_corrected       False
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [51]:
df = df.replace({'humour': {'not_funny': 0, 'funny': 1, 'very_funny': 2, 'hilarious':3},
            'sarcasm': {'not_sarcastic': 0, 'general': 1, 'twisted_meaning': 2, 'very_twisted': 3},
            'offensive': {'not_offensive': 0, 'slight': 1, 'very_offensive': 2, 'hateful_offensive': 3},
            'motivational': {'not_motivational': 0, 'motivational': 1},
            'overall_sentiment': {'very_negative': 0, 'negative': 1, 'neutral': 2, 'positive': 3, 'very_positive': 4}})

In [52]:
from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [53]:
width = 100
height = 100
X_full = []
X_full_path = []
for i in tqdm(range(full_df.shape[0])):
    path = '/content/memotion_dataset_7k/images/'+full_df['image_name'][i]
    img = image.load_img(path,target_size=(width,height,3))
    img = image.img_to_array(img)
    img = img/255.0
    X_full.append(img)
    X_full_path.append(path)

 93%|█████████▎| 6507/6992 [00:39<00:02, 185.36it/s]/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
 95%|█████████▌| 6659/6992 [00:39<00:01, 199.20it/s]/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 6992/6992 [00:42<00:00, 164.93it/s]


In [54]:
width = 100
height = 100
X_cleaned = []
X_cleaned_path = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X_cleaned.append(img)
        X_cleaned_path.append(path)

100%|██████████| 6987/6987 [00:44<00:00, 158.36it/s]


In [55]:
deleted_paths = [paths for paths in X_full_path + X_cleaned_path if paths not in X_full_path or paths not in X_cleaned_path]
deleted_paths

['/content/memotion_dataset_7k/images/image_120.jpg',
 '/content/memotion_dataset_7k/images/image_4800.jpg',
 '/content/memotion_dataset_7k/images/image_6782.jpg',
 '/content/memotion_dataset_7k/images/image_6785.jpg',
 '/content/memotion_dataset_7k/images/image_6787.jpg',
 '/content/memotion_dataset_7k/images/image_6988.jpg',
 '/content/memotion_dataset_7k/images/image_6989.jpg',
 '/content/memotion_dataset_7k/images/image_6990.png',
 '/content/memotion_dataset_7k/images/image_6991.jpg',
 '/content/memotion_dataset_7k/images/image_6992.jpg']

In [58]:
delete_var = [full_df, clean_df, X_cleaned, X_full, X_cleaned_path, X_full_path]
for i in delete_var:
    del i

In [63]:
len(clean_df)

6987

In [60]:
width = 100
height = 100
X = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X.append(img)

100%|██████████| 6987/6987 [00:43<00:00, 159.72it/s]


In [64]:
X = np.array(X)

In [65]:
X.shape

(6982, 100, 100, 3)

In [66]:
rows_to_drop = ['image_120.jpg',
              'image_4800.jpg',
              'image_6782.jpg',
              'image_6785.jpg',
              'image_6787.jpg',
              'image_6988.jpg',
              'image_6989.jpg',
              'image_6990.png',
              'image_6991.jpg',
              'image_6992.jpg']

In [67]:
cleaner_df = df
cleaner_df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,3,1,0,0,4
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,0,1,0,1,4
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,2,0,0,0,3
3,image_4.png,10 Year Challenge - Sweet Dee Edition,2,2,2,1,3
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,3,3,2,0,2


In [68]:
len(cleaner_df)

6992

In [69]:
for images in rows_to_drop:
    cleaner_df.drop(cleaner_df[cleaner_df['image_name'] == images].index, inplace=True)

In [70]:
cleaner_df.shape

(6982, 7)

In [71]:
np.save('image_array', X)

In [72]:
Y = cleaner_df.iloc[:,2:]
Y.shape

(6982, 5)

In [74]:
X.shape, Y.shape

((6982, 100, 100, 3), (6982, 5))

In [75]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.resnet_v2.preprocess_input

rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [76]:
base_model_1 = tf.keras.applications.ResNet50(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')
base_model_2 = tf.keras.applications.VGG16(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')

In [77]:
base_model_1.trainable = False
base_model_2.trainable = False

In [78]:
global_average_layer = GlobalAveragePooling2D()

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

In [80]:
del X

In [81]:
image_input = tf.keras.Input(shape=(100, 100, 3), name = 'image_input')
iamge_layers = data_augmentation(image_input)
iamge_layers = preprocess_input(iamge_layers)
layer_bm_1 = base_model_1(image_input, training=False)
layer_bm_1 = Conv2D(2048, kernel_size=2,padding='valid')(layer_bm_1)
layer_bm_1 = Dense(512)(layer_bm_1)
layer_bm_2 = base_model_2(image_input, training=False)
layer_bm_2 = Dense(512)(layer_bm_2)
layers = tf.keras.layers.concatenate([layer_bm_1, layer_bm_2])
iamge_layers = global_average_layer(layer_bm_1)
image_layers = Dropout(0.2, name = 'dropout_layer')(iamge_layers)

In [82]:
cleaner_df.shape

(6982, 7)

In [83]:
def standardization(data):
    data = data.apply(lambda x: x.lower())
    data = data.apply(lambda x: re.sub(r'\d+', '', x))
    data = data.apply(lambda x: re.sub(r'.com', '', x, flags=re.MULTILINE))
    data = data.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    return data

df['text_corrected'] = standardization(cleaner_df.text_corrected)

In [84]:
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(cleaner_df.text_corrected, Y, test_size = 0.2, random_state=42)

In [86]:
X_text_train

6568      how much money mark zuckerberg gave out imgflip
4004    crypto experts choose crypto investments based...
1011    me throwing away all the leonardo dicaprio mem...
4618    texan man searching for supplies and dry land ...
932     keep calm and use duct tape if you cant fix it...
                              ...                        
3773           let me tell you why thats bullshit imgflip
5193    and thats how mafia works the godfather mobile...
5228                              huh so babies cant swim
5392    leonard hofstadter a guy who actually made it ...
861     if you dont like me because im country just re...
Name: text_corrected, Length: 5585, dtype: object

In [87]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

text_ds = np.asarray(X_text_train)
vectorize_layer.adapt(tf.convert_to_tensor(text_ds))

In [88]:
embedding_dim=16

text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
text_layers = vectorize_layer(text_input)
text_layers = tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding")(text_layers)
text_layers = tf.keras.layers.Dropout(0.5)(text_layers)

text_layers = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='relu', return_sequences=True))(text_layers)

text_layers = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(text_layers)
text_layers = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(text_layers)
text_layers = tf.keras.layers.GlobalMaxPooling1D()(text_layers)

# We add a vanilla hidden layer:
text_layers = tf.keras.layers.Dense(2048, activation="relu")(text_layers)
text_layers = tf.keras.layers.Dropout(0.5)(text_layers)

In [89]:
concatenate = tf.keras.layers.concatenate([image_layers, text_layers], axis=1)

In [90]:
overall_layer = tf.keras.layers.Dense(2048, activation='softmax')(concatenate)

In [91]:
prediction_layer_1 = tf.keras.layers.Dense(4, activation='softmax', name = 'sarcasm')
prediction_layer_2 = tf.keras.layers.Dense(4, activation='softmax', name = 'humuor')
prediction_layer_3 = tf.keras.layers.Dense(4, activation='softmax', name = 'offensive')
prediction_layer_4 = tf.keras.layers.Dense(1, activation='sigmoid', name = 'motivational')
prediction_layer_5 = tf.keras.layers.Dense(5, activation='softmax', name = 'overall')

In [92]:
output_1 = prediction_layer_1(overall_layer)
output_2 = prediction_layer_2(overall_layer)
output_3 = prediction_layer_3(overall_layer)
output_4 = prediction_layer_4(overall_layer)
output_5 = prediction_layer_5(overall_layer)
model = tf.keras.Model(inputs = [image_input, text_input] , outputs = [output_1, output_2, output_3, output_4, output_5])

In [94]:
import tensorflow_addons as tfa

In [95]:
base_learning_rate = 0.01
losses = {
      "humuor": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "sarcasm": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "offensive": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "motivational": tf.keras.losses.BinaryCrossentropy(from_logits=False),
      "overall": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
}
lossWeights = {
      "humuor": 1.0, 
      "sarcasm": 1.0, 
      "offensive": 1.0, 
      "motivational": 1.0,
      "overall": 1.0
}
metric = {
    "humuor": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "sarcasm": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "offensive": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "motivational": ['acc',tfa.metrics.F1Score(num_classes=1, average="micro", threshold = 0.9)],
    "overall": ['acc',tfa.metrics.F1Score(num_classes=5, average="micro", threshold = 0.9)]
}
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
#               loss = losses,
#               loss_weights= lossWeights,
#               metrics=metric)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [113]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model-{epoch:03d}-{overall_accuracy:03f}-{val_overall_accuracy:03f}', save_format='tf', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=27, verbose=1, epsilon=1e-4, mode='min')

In [116]:
learning_rates = [0.001] # [0.1, 0.01, 0.001, 0.0001, 0.00001]
for i in learning_rates:
    model.compile(optimizer=tf.keras.optimizers.RMSprop(i),
              loss = losses,
              loss_weights= lossWeights,
              metrics=['accuracy'])
    
    history = model.fit(x = {"image_input": X_train, "text": X_text_train},
                    y = {"sarcasm": y_train.sarcasm, 
                         "humuor": y_train.humour, 
                         "offensive": y_train.offensive, 
                         "motivational": y_train.motivational, 
                         "overall": y_train.overall_sentiment},
                    batch_size=64,
                    epochs=50,
                    verbose=2,
                    validation_split=0.2,
                    callbacks=[earlyStopping, mcp_save,reduce_lr_loss]
                   )

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


70/70 - 38s - loss: 5.5813 - sarcasm_loss: 1.1839 - humuor_loss: 1.2894 - offensive_loss: 1.1727 - motivational_loss: 0.6486 - overall_loss: 1.2869 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.4009 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465 - val_loss: 5.5441 - val_sarcasm_loss: 1.1542 - val_humuor_loss: 1.2963 - val_offensive_loss: 1.1800 - val_motivational_loss: 0.6591 - val_overall_loss: 1.2545 - val_sarcasm_accuracy: 0.5157 - val_humuor_accuracy: 0.3509 - val_offensive_accuracy: 0.3724 - val_motivational_accuracy: 0.6321 - val_overall_accuracy: 0.4628 - lr: 0.0010 - 38s/epoch - 543ms/step
Epoch 2/50
70/70 - 29s - loss: 5.5815 - sarcasm_loss: 1.1849 - humuor_loss: 1.2885 - offensive_loss: 1.1748 - motivational_loss: 0.6485 - overall_loss: 1.2849 - sarcasm_accuracy: 0.4944 - humuor_accuracy: 0.3581 - offensive_accuracy: 0.3906 - motivational_accuracy: 0.6486 - overall_accuracy: 0.4465 - val_loss: 5.5427 - val_sarcasm_loss: 1.1543 - val_

In [117]:
evaluate = model.evaluate(x = {"image_input": X_test, "text": X_text_test},
                    y = {"sarcasm": y_test.sarcasm, 
                         "humuor": y_test.humour, 
                         "offensive": y_test.offensive, 
                         "motivational": y_test.motivational, 
                         "overall": y_test.overall_sentiment},
                    batch_size=32,
                    verbose=2
                   )

44/44 - 2s - loss: 5.5598 - sarcasm_loss: 1.1697 - humuor_loss: 1.3106 - offensive_loss: 1.1733 - motivational_loss: 0.6440 - overall_loss: 1.2623 - sarcasm_accuracy: 0.5140 - humuor_accuracy: 0.3278 - offensive_accuracy: 0.3844 - motivational_accuracy: 0.6557 - overall_accuracy: 0.4359 - 2s/epoch - 54ms/step
